In [1]:
import pandas as pd
import numpy as np 

ENDERECO_DADOS = 'https://www.ispdados.rj.gov.br/Arquivos/BaseDPEvolucaoMensalCisp.csv'

try:
    print('Carregando os dados...')
    df_ocorrencia = pd.read_csv(ENDERECO_DADOS, sep=';', encoding='iso-8859-1')

    print(df_ocorrencia.head())

    print('Dados carregados com sucesso!')
except Exception as e:
    print(f'Erro ao carregar os dados: {e}')
    exit()

Carregando os dados...
   cisp  mes   ano  mes_ano  aisp  risp           munic    mcirc   regiao  \
0     1    1  2003  2003m01     5     1  Rio de Janeiro  3304557  Capital   
1     4    1  2003  2003m01     5     1  Rio de Janeiro  3304557  Capital   
2     5    1  2003  2003m01     5     1  Rio de Janeiro  3304557  Capital   
3     6    1  2003  2003m01     1     1  Rio de Janeiro  3304557  Capital   
4     7    1  2003  2003m01     1     1  Rio de Janeiro  3304557  Capital   

   hom_doloso  ...  cmp  cmba  ameaca  pessoas_desaparecidas  \
0           0  ...  NaN   NaN      21                      2   
1           3  ...  NaN   NaN      15                      6   
2           3  ...  NaN   NaN      47                      2   
3           6  ...  NaN   NaN      26                      2   
4           4  ...  NaN   NaN      10                      1   

   encontro_cadaver  encontro_ossada  pol_militares_mortos_serv  \
0                 0                0                          

In [4]:
#recebendo dados 

try: 

    df_veiculos = df_ocorrencia[['cisp', 'roubo_veiculo', 'recuperacao_veiculos']] 

    df_total_veiculos = df_veiculos.groupby('cisp').sum(['roubo', 'recuperacao_veiculos']).reset_index() 

    print(df_total_veiculos) 

 
except Exception as e: 

    print(f'Erro: {e}') 

    exit() 


     cisp  roubo_veiculo  recuperacao_veiculos
0       1            565                   536
1       4           2165                  1903
2       5           1316                  1546
3       6           4516                  7974
4       7           1997                  1824
..    ...            ...                   ...
133   159            283                   338
134   165            500                   647
135   166            968                  1487
136   167            105                   399
137   168             62                   165

[138 rows x 3 columns]


In [5]:
try:
    df_total_veiculos_cut = (
        df_total_veiculos[df_total_veiculos['roubo_veiculo'] < np.percentile(df_veiculos['roubo_veiculo'], 95)]
    )

    df_total_veiculos_cut = (
        
        df_total_veiculos_cut[df_total_veiculos_cut['recuperacao_veiculos'] < np.percentile(df_veiculos['recuperacao_veiculos'], 99)]
    )

#array
    
    array_roubo_veiculo = np.array(df_total_veiculos_cut['roubo_veiculo'])

    array_recuperacao_veiculo = np.array(df_total_veiculos_cut['recuperacao_veiculos'])

    correlacao = np.corrcoef(array_roubo_veiculo, array_recuperacao_veiculo)[0, 1]

    print(f'Correlação: {correlacao:.2f}')

except Exception as e:
    print(f'Erro ao processar os dados cut: {e}')
    exit()

Correlação: 0.24


In [6]:
try:
     # pip install scikit-learn
    print('Iniciando a regressão linear...')

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        array_roubo_veiculo,  # Variável independente
        array_recuperacao_veiculo,  # Variável dependente
        test_size=0.2,  # 80% treino, 20% teste
        random_state=42  # Para reprodutibilidade
    )

    # Normalizando os dados
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, 1))
    X_test = scaler.transform(X_test.reshape(-1, 1))

    # Treinando o modelo
    from sklearn.linear_model import LinearRegression
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)  # Treinando

    r2_score = modelo.score(X_test, y_test)  # Avaliando
    print(f"R² score: {r2_score:.2f}")
except Exception as e:
    print(f'Erro ao realizar a regressão linear: {e}')
    exit()

Iniciando a regressão linear...
R² score: -0.26


In [ ]:
try:
    # Array de provisões 35000, 43000, 50000

    array_roubo_veiculo_pred = np.array([35000, 43000, 50000])
    array_roubo_veiculo_pred_scaled = scaler.transform(array_roubo_veiculo_pred.reshape(-1, 1))

    recup_pred = modelo.predict(array_roubo_veiculo_pred_scaled)
    print("Provisões de recuperação de veículos: ", recup_pred)

except Exception as e:
    print(f'Erro ao fazer previsões: {e}')
    exit()
